In [1]:
!git --version


git version 2.34.1


In [3]:
!git config --global user.name "nithisri05"
!git config --global user.email "nithisri.arunkumar@gmail.com"


In [4]:
%cd /content


/content


In [5]:
!mkdir fdic-regulatory-loan-assistant
%cd fdic-regulatory-loan-assistant


/content/fdic-regulatory-loan-assistant


#Prompt Engineering for Regulatory-Aligned Loan Evaluation

FDIC RMS Manual – Section 3.2 (Loans)

#Problem Statement

Banks must evaluate loan applications under strict regulatory supervision.
This project explores how prompt engineering can be used to constrain a Large Language Model (LLM) to provide accurate, examiner-aligned regulatory reasoning based only on FDIC RMS Manual Section 3.2, without making loan decisions or hallucinating unsupported rules.

#Proposed Method

* Use prompt engineering only (no model training or fine-tuning)

* Encode regulatory guidance into a structured constraints object

* Enforce behavior using a strict system prompt

* Separate data extraction from regulatory reasoning

* Explicitly refuse prohibited requests (e.g., loan approval decisions)

#Model Used & Context Window

* Model: gpt-4.1-nano

* Context Window: ~128,000 tokens

* Status: All prompts, constraints, and inputs are well within the context limit, ensuring no truncation and stable regulatory behavior.

#What This Notebook Contains

* A structured representation of FDIC Section 3.2

* Prompt-controlled extraction of loan data from text or images

* A regulatory reasoning assistant constrained to examiner expectations

* A professional chatbot interface for interaction

* Explicit safety and refusal mechanisms

#Regulatory Constraints Object

Defines FDIC_SECTION_3_2_CONSTRAINTS, the single source of truth containing governance, underwriting, repayment analysis, collateral, loan review expectations, prohibitions, and scope rules.

In [ ]:
FDIC_SECTION_3_2_CONSTRAINTS = {

    "meta": {
        "document": "FDIC RMS Manual of Examination Policies – Section 3.2 (Loans)",
        "regulatory_intent": "Supervisory guidance for safe and sound lending practices",
        "scope_limitation": "Provides regulatory considerations, not lending decisions",
        "single_source_of_truth": True
    },

    # --------------------------------------------------
    # 1. GOVERNANCE & OVERSIGHT CONSTRAINTS
    # --------------------------------------------------
    "governance_constraints": {
        "board_responsibility": [
            "Board of directors must approve and oversee lending policies",
            "Board must review lending policies periodically",
            "Board retains ultimate responsibility for loan portfolio quality"
        ],
        "management_responsibility": [
            "Management must implement board-approved lending policies",
            "Management must ensure adherence to policies and procedures",
            "Management must identify, monitor, and address credit risk"
        ]
    },

    # --------------------------------------------------
    # 2. LENDING POLICY CONSTRAINTS
    # --------------------------------------------------
    "lending_policy_constraints": {
        "policy_requirements": [
            "Written and up-to-date lending policies are required",
            "Policies must reflect institution size, complexity, and market",
            "Policies must not be static and must adapt to changing conditions"
        ],
        "policy_coverage": [
            "Types of loans permitted",
            "Lending authority limits",
            "Collateral requirements",
            "Loan documentation standards",
            "Collection and workout procedures",
            "Portfolio diversification goals",
            "Loan review and grading systems",
            "Allowance for credit losses governance",
            "Environmental risk safeguards"
        ]
    },

    # --------------------------------------------------
    # 3. UNDERWRITING & CREDIT ANALYSIS CONSTRAINTS
    # --------------------------------------------------
    "underwriting_constraints": {
        "repayment_analysis": [
            "Primary sources of repayment must be identified",
            "Secondary sources of repayment must be realistic and documented",
            "Reliance on refinancing alone is a supervisory concern"
        ],
        "cash_flow_expectations": [
            "Cash flow assumptions must be reasonable and supportable",
            "Overly optimistic projections are discouraged",
            "Borrower capacity to service debt must be evaluated qualitatively"
        ],
        "loan_structure": [
            "Loan terms must align with risk characteristics",
            "Covenants should be appropriate to the loan type",
            "Collateral structure should support orderly repayment"
        ]
    },

    # --------------------------------------------------
    # 4. COLLATERAL & VALUATION CONSTRAINTS
    # --------------------------------------------------
    "collateral_constraints": {
        "general": [
            "Collateral must be properly documented",
            "Lien perfection must be verified",
            "Collateral valuation must be appropriate and supportable"
        ],
        "valuation_principles": [
            "Valuations must reflect current market conditions",
            "Independent valuations are preferred for higher-risk credits",
            "Enterprise value reliance must be clearly justified"
        ],
        "limitations": [
            "Collateral liquidation is not a primary repayment strategy",
            "Illiquid or hard-to-value collateral requires additional scrutiny"
        ]
    },

    # --------------------------------------------------
    # 5. LOAN REVIEW SYSTEM CONSTRAINTS
    # --------------------------------------------------
    "loan_review_constraints": {
        "system_requirements": [
            "All institutions must maintain an effective loan review system",
            "Loan review systems must identify emerging credit weaknesses",
            "Loan review findings must be reported to senior management"
        ],
        "independence": [
            "Independent loan review is preferred",
            "Conflicts of interest must be avoided",
            "Loan officers’ judgments should be subject to review"
        ],
        "frequency": [
            "Significant loans must be reviewed at least annually",
            "Higher-risk loans require more frequent review"
        ]
    },

    # --------------------------------------------------
    # 6. CREDIT GRADING & RISK IDENTIFICATION CONSTRAINTS
    # --------------------------------------------------
    "credit_grading_constraints": {
        "grading_principles": [
            "Credit grades must reflect current conditions",
            "Grades must be updated when credit quality changes",
            "Grades should align with regulatory classification frameworks"
        ],
        "problem_identification": [
            "Loans with well-defined weaknesses must be identified promptly",
            "Problem loans require heightened monitoring",
            "Adverse trends must be escalated to management"
        ]
    },

    # --------------------------------------------------
    # 7. ALLOWANCE FOR CREDIT LOSSES CONSTRAINTS
    # --------------------------------------------------
    "credit_loss_constraints": {
        "general": [
            "Institutions must maintain an appropriate allowance for credit losses",
            "Allowance must reflect current conditions and reasonable forecasts",
            "Allowance methodology must be well-documented and consistent"
        ],
        "analysis_requirements": [
            "Historical loss experience alone is insufficient",
            "Qualitative factors must be considered",
            "Portfolio segmentation is required for collective evaluation"
        ],
        "governance": [
            "Board and management are responsible for allowance adequacy",
            "Allowance must be evaluated at least quarterly"
        ]
    },

    # --------------------------------------------------
    # 8. LOAN TYPE–SPECIFIC SENSITIVITY CONSTRAINTS
    # --------------------------------------------------
    "loan_type_sensitivity": {
        "commercial_loans": [
            "Borrower financial condition must be analyzed",
            "Working capital and term purposes must align with structure"
        ],
        "commercial_real_estate": [
            "Repayment reliance on property cash flow is emphasized",
            "Vacancy, market conditions, and appraisal quality are relevant"
        ],
        "leveraged_lending": [
            "Ability to de-lever over time is a central concern",
            "Enterprise value reliance requires heightened scrutiny",
            "Stress scenarios and sponsor support are considered"
        ],
        "agricultural_loans": [
            "Commodity price sensitivity is relevant",
            "Seasonality and production cycles must be considered"
        ],
        "consumer_loans": [
            "Homogeneous loan pooling is appropriate",
            "Volume trends and delinquency patterns are relevant"
        ]
    },

    # --------------------------------------------------
    # 9. RISK INDICATOR CONSTRAINTS
    # --------------------------------------------------
    "risk_indicator_constraints": {
        "credit_level": [
            "Incomplete or outdated credit information",
            "Weak repayment capacity",
            "Excessive reliance on collateral",
            "Covenant violations or waivers"
        ],
        "portfolio_level": [
            "Concentration risk",
            "Rapid growth without adequate controls",
            "Sensitivity to economic or industry downturns"
        ]
    },

    # --------------------------------------------------
    # 10. EXAMINER EXPECTATION CONSTRAINTS
    # --------------------------------------------------
    "examiner_expectations": {
        "focus": [
            "Process quality over outcomes",
            "Risk identification over risk elimination",
            "Documentation and governance effectiveness"
        ],
        "acceptance": [
            "Judgment-based estimates are acceptable when well-supported",
            "Different institutions may use different methodologies"
        ]
    },

    # --------------------------------------------------
    # 11. HARD PROHIBITIONS
    # --------------------------------------------------
    "explicit_prohibitions": [
        "No loan approval or rejection decisions",
        "No borrower eligibility determinations",
        "No credit scoring or ranking",
        "No numerical thresholds or ratios",
        "No pricing or profitability analysis"
    ],

    # --------------------------------------------------
    # 12. RESPONSE BEHAVIOR CONSTRAINTS
    # --------------------------------------------------
    "response_constraints": {
        "tone": [
            "Professional",
            "Neutral",
            "Regulatory",
            "Examiner-oriented"
        ],
        "required_disclaimer": (
            "This response provides regulatory considerations only and does not "
            "constitute a credit decision."
        )
    },

    # --------------------------------------------------
    # 13. QUESTION INTERPRETATION & SCOPE CONSTRAINTS
    # --------------------------------------------------
    "question_scope_constraints": {
        "always_in_scope_topics": [
            "repayment analysis",
            "primary and secondary sources of repayment",
            "borrower repayment capacity",
            "cash flow reasonableness",
            "underwriting analysis related to repayment",
            "documentation supporting repayment assumptions"
        ],
        "interpretation_rules": [
            "Questions referencing repayment analysis or sources of repayment are always within scope under FDIC RMS Manual Section 3.2",
            "Such questions must be answered by describing supervisory expectations and examiner focus, not by evaluating loan approval",
            "Insufficient loan detail should be acknowledged without refusing the question",
            "Responses must emphasize process quality and risk identification rather than outcomes"
        ],
        "refusal_guardrails": [
            "Repayment-related questions must not be refused solely due to loan specificity",
            "Refusal is appropriate only when the question is unrelated to banking, lending, or financial regulation"
        ]
    }
}


#Image-Based Loan Extraction

Extracts visible loan information from uploaded images into structured JSON without inference or calculation.

In [ ]:
from openai import OpenAI
import json
import base64

client = OpenAI(
    api_key="API_KEY",
    base_url="BASE_URL/"
)

def image_to_base64(image_path):
    with open(image_path, "rb") as img:
        return base64.b64encode(img.read()).decode("utf-8")

def extract_from_image(image_path):
    image_base64 = image_to_base64(image_path)

    system_prompt = """
You are a data extraction assistant.

Extract information ONLY from the loan application image.
Do NOT infer, calculate, or guess.
If information is missing or unclear, use null.

Return ONLY valid JSON in this format:

{
  "loan": {
    "type": null,
    "amount": null,
    "interest_rate": null,
    "term_months": null,
    "amortization": null
  },
  "property": {
    "address": null,
    "county": null,
    "property_use": null
  },
  "purpose": null,
  "borrower": {
    "name": null,
    "age": null,
    "marital_status": null
  },
  "employment": {
    "employer": null,
    "position": null,
    "years_on_job": null
  }
}
"""

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{image_base64}"
                        }
                    }
                ]
            }
        ],
        temperature=0
    )

    return json.loads(response.choices[0].message.content)


#Text-Based Loan Extraction

Extracts loan details from text summaries using the same schema as image extraction.

In [ ]:
image_path = "/content/Loan_application.jpeg"

result = extract_from_image(image_path)

print(json.dumps(result, indent=2))


{
  "loan": {
    "type": "Conventional",
    "amount": 180000,
    "interest_rate": 10.49,
    "term_months": 360,
    "amortization": "Fixed Rate"
  },
  "property": {
    "address": "0000 NW 45TH STREET, MIAMI, FL 33127",
    "county": "MIAMI-DADE",
    "property_use": "Primary Residence"
  },
  "purpose": "Purchase",
  "borrower": {
    "name": "JOHN SAMPLE",
    "age": 53,
    "marital_status": "Unmarried"
  },
  "employment": {
    "employer": "POLICE DEPARTMENT",
    "position": "CLIENT RELATIONS LEGAL SERVICES",
    "years_on_job": 7
  }
}


#System Prompt (Behavior Control)

A strict system prompt that enforces:

* FDIC Section 3.2 grounding

* Examiner-oriented tone

* Question-specific answers

* Mandatory refusal of loan approvals

* Mandatory answers for repayment analysis questions

In [ ]:
FDIC_SYSTEM_PROMPT = """
You are a Regulatory Reasoning Assistant operating strictly under the
FDIC RMS Manual of Examination Policies – Section 3.2 (Loans).

You are governed by a structured constraint object named:
FDIC_SECTION_3_2_CONSTRAINTS

FDIC_SECTION_3_2_CONSTRAINTS is your SOLE source of regulatory truth.

--------------------------------------------------
ROLE & REGULATORY CONTEXT
--------------------------------------------------
Your role is to explain how FDIC examiners assess lending practices
and credit risk management processes under Section 3.2.

You do NOT make credit decisions.
You do NOT approve or reject loans.

--------------------------------------------------
CRITICAL OVERRIDE: REPAYMENT ANALYSIS (MANDATORY)
--------------------------------------------------
Questions related to repayment analysis, sources of repayment, borrower
repayment capacity, or cash flow MUST ALWAYS BE ANSWERED.

You are REQUIRED to describe examiner expectations and supervisory
focus areas for repayment analysis.

You MUST NOT:
- Refuse repayment-related questions
- State that repayment analysis cannot be evaluated
- State that specific concerns cannot be discussed

Discussing examiner concerns related to repayment analysis is NOT a
loan approval, NOT a credit decision, and NOT prohibited.

--------------------------------------------------
DECISION QUESTION HANDLING
--------------------------------------------------
If the question asks whether a loan should be approved, rejected,
or granted, you MUST:

1. Explicitly state that FDIC Section 3.2 does not establish approval
   criteria and that this system cannot approve or reject loans.
2. AFTER THAT, explain examiner expectations regarding processes
   such as underwriting, repayment analysis, documentation, and
   monitoring.

--------------------------------------------------
HOW TO ANSWER QUESTIONS
--------------------------------------------------
You MUST:
- Answer the user’s question directly
- Limit your discussion strictly to the topic asked
- Frame all analysis from an examiner perspective
- Focus on process quality, documentation, and risk identification

You MUST NOT:
- Shift to unrelated governance topics unless directly relevant
- Generalize beyond the user’s question
- Use refusal language for in-scope topics

--------------------------------------------------
SOURCE OF TRUTH ENFORCEMENT
--------------------------------------------------
Use ONLY concepts explicitly present in FDIC RMS Manual Section 3.2:
- Primary and secondary repayment sources
- Reasonableness of repayment assumptions
- Avoidance of speculative or refinancing-only repayment
- Documentation supporting repayment capacity
- Examiner focus on process, not outcomes

--------------------------------------------------
RESPONSE STYLE
--------------------------------------------------
Write in professional, examiner-oriented prose.
Use paragraphs, not bullet points.
Do not hedge or over-disclaim.

--------------------------------------------------
REQUIRED DISCLAIMER (VERBATIM)
--------------------------------------------------
"This response provides regulatory considerations only and does not
constitute a loan approval, rejection, or credit decision."

"""


#Regulatory Reasoning Function

Generates question-focused regulatory analysis using only FDIC Section 3.2 and the provided loan data.

In [ ]:
def fdic_regulatory_analysis(loan_json, user_question):
    user_prompt = f"""
Loan Application Data:
{json.dumps(loan_json, indent=2)}

User Question:
{user_question}

Answer ONLY the user's question using FDIC RMS Manual
Section 3.2 and FDIC_SECTION_3_2_CONSTRAINTS.
"""

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": FDIC_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    return response.choices[0].message.content


In [ ]:
analysis_output = fdic_regulatory_analysis(
    result,
    "What regulatory considerations apply to this loan under FDIC RMS Section 3.2?"
)
print(analysis_output)


Under FDIC RMS Manual Section 3.2, the regulatory considerations for this loan primarily focus on the adequacy of the borrower's repayment capacity, the quality of the repayment sources, and the documentation supporting those sources. Examiners assess whether the primary repayment source—typically the borrower's income from employment—is sufficient and reliable to service the debt over the loan term. In this case, the borrower's employment with the Police Department as a Client Relations position, with seven years on the job, provides a stable income stream that should be evaluated for reasonableness and consistency.

Examiners also scrutinize secondary repayment sources, if any are identified, to ensure they are credible and appropriately documented. The loan's purpose, a purchase, is a common use, but the examiner will verify that the documentation supports the borrower's ability to repay based on their financial profile. The interest rate, loan amount, and term are also considered i

#Loan Summary Formatter

Converts extracted JSON into a clean, human-readable loan summary for transparency.

In [ ]:
def format_loan_data(loan_json):
    def show(val):
        if val is None or val == "" or val == "null":
            return "Not Provided"
        return val

    loan = loan_json.get("loan", {})
    prop = loan_json.get("property", {})
    borrower = loan_json.get("borrower", {})
    employment = loan_json.get("employment", {})

    formatted = f"""
📄 LOAN APPLICATION – EXTRACTION SUMMARY

────────────────────────────────────
LOAN INFORMATION
────────────────────────────────────
Loan Type          : {show(loan.get("type"))}
Loan Amount        : {show(loan.get("amount"))}
Interest Rate      : {show(loan.get("interest_rate"))}
Loan Term (Months) : {show(loan.get("term_months"))}
Amortization       : {show(loan.get("amortization"))}

────────────────────────────────────
PROPERTY INFORMATION
────────────────────────────────────
Address            : {show(prop.get("address"))}
County             : {show(prop.get("county"))}
Property Use       : {show(prop.get("property_use"))}

────────────────────────────────────
BORROWER INFORMATION
────────────────────────────────────
Name               : {show(borrower.get("name"))}
Age                : {show(borrower.get("age"))}
Marital Status     : {show(borrower.get("marital_status"))}

────────────────────────────────────
EMPLOYMENT INFORMATION
────────────────────────────────────
Employer           : {show(employment.get("employer"))}
Position           : {show(employment.get("position"))}
Years on Job       : {show(employment.get("years_on_job"))}

✔ Extraction completed successfully.
"""
    return formatted.strip()


In [ ]:
def extract_from_text(text):
    system_prompt = """
You are a data extraction assistant.

Extract loan details from the given text.
Do NOT infer missing information.
If information is missing, use null.
Return ONLY valid JSON using the same schema as image extraction.
"""

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text}
        ],
        temperature=0
    )

    return json.loads(response.choices[0].message.content)


#Scope Classification Logic

Determines whether a question is within FDIC loan supervision scope and prevents unnecessary refusals.

In [ ]:
FDIC_SCOPE_CLASSIFIER_PROMPT = """
You are a regulatory scope classifier operating in a banking
supervisory context.

Your task is to determine whether the user's question can be
reasonably answered using supervisory and regulatory guidance
from the FDIC RMS Manual of Examination Policies – Section 3.2 (Loans).

IMPORTANT INTERPRETATION RULES (MANDATORY):

1. The presence of a loan application means the default context
   is loan supervision unless clearly disproven.

2. Questions using terms such as:
   - regulatory considerations
   - regulations to be followed
   - requirements
   - specifications
   - compliance expectations
   - supervisory focus
   - examiner concerns

   MUST be interpreted as referring to FDIC loan supervision
   under Section 3.2, even if the words "examiner" or
   "loan supervision" are not explicitly stated.

3. Questions referring to:
   - "this loan"
   - "this application"
   - "applying this loan"
   - "processing this loan"

   MUST be treated as loan-specific and therefore IN SCOPE.

4. Generic FDIC questions are IN SCOPE IF they can be reasonably
   answered by discussing supervisory expectations, governance,
   underwriting discipline, risk management, collateral,
   documentation, or loan review processes.

5. ONLY classify a question as OUT_OF_SCOPE if it is clearly
   unrelated to banking, lending, credit risk, loan supervision,
   or financial regulation (e.g., medical, personal, technical,
   entertainment topics).

You are NOT evaluating answer quality.
You are ONLY evaluating domain relevance.

Return EXACTLY one of the following outputs:

IN_SCOPE
OUT_OF_SCOPE

Do NOT explain.
Do NOT add any additional text.

"""


#Controller Function

Coordinates extraction, formatting, scope validation, and regulatory reasoning to ensure consistent behavior.

In [ ]:
def fdic_scope_gate(user_question: str, loan_json: dict) -> str:
    """
    Context-aware three-state scope gate.
    Returns one of: IN_SCOPE, AMBIGUOUS, OUT_OF_SCOPE
    """

    gate_prompt = f"""
Loan Application Context:
{json.dumps(loan_json, indent=2)}

User Question:
{user_question}
"""

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": FDIC_SCOPE_CLASSIFIER_PROMPT},
            {"role": "user", "content": gate_prompt}
        ],
        temperature=0
    )

    decision = response.choices[0].message.content.strip()

    # Debug (keep during testing)
    print("SCOPE GATE DECISION:", decision)

    if decision not in {"IN_SCOPE", "AMBIGUOUS", "OUT_OF_SCOPE"}:
        return "OUT_OF_SCOPE"

    return decision


In [ ]:
def fdic_regulatory_analysis(loan_json, user_question):
    user_prompt = f"""
Loan Application Data:
{json.dumps(loan_json, indent=2)}

User Question:
{user_question}

Answer ONLY the user's question using FDIC RMS Manual
Section 3.2 and FDIC_SECTION_3_2_CONSTRAINTS.
"""

    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[
            {"role": "system", "content": FDIC_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )

    return response.choices[0].message.content


In [ ]:
def process_user_input(text_input, image_input, question_input):

    # Step 1: Extract loan data
    if image_input:
        loan_json = extract_from_image(image_input)
    elif text_input and text_input.strip():
        loan_json = extract_from_text(text_input)
    else:
        return (
            "❌ Please provide a loan application (text or image).",
            ""
        )

    structured_output = format_loan_data(loan_json)

    # Step 2: Validate question presence
    if not question_input or question_input.strip() == "":
        return (
            structured_output,
            "❌ Please enter a regulatory question related to the loan."
        )

    # Step 3: Scope classification
    scope_decision = fdic_scope_gate(question_input, loan_json)

    # Step 4: Handle scope outcomes
    if scope_decision == "OUT_OF_SCOPE":
        return (
            structured_output,
            "I’m unable to answer that question. This system is limited to "
            "regulatory and supervisory considerations related to lending practices "
            "under the FDIC RMS Manual of Examination Policies, Section 3.2."
        )

    if scope_decision == "AMBIGUOUS":
        return (
            structured_output,
            "Your question refers to regulations or requirements but does not "
            "specify loan supervision. Could you clarify whether you are asking "
            "about regulatory or supervisory considerations for the provided loan "
            "under FDIC RMS Manual Section 3.2?"
        )

    # Step 5: IN_SCOPE → regulatory reasoning
    regulatory_output = fdic_regulatory_analysis(
        loan_json,
        question_input
    )

    return structured_output, regulatory_output


#Gradio Chat Interface

Provides a professional chatbot UI that accepts loan input and regulatory questions and displays structured outputs.

In [ ]:
import gradio as gr
with gr.Blocks(
    title="FDIC Regulatory Loan Reasoning Assistant",
    theme=gr.themes.Soft()
) as demo:

    gr.Markdown("""
    ## 🏦 FDIC Regulatory Loan Reasoning Assistant
    **Regulatory Context:** FDIC RMS Manual of Examination Policies – Section 3.2 (Loans)

    This system provides **regulatory and supervisory interpretation only**.
    It **does not approve, reject, or score loans**.
    """)

    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(
                label="Loan Application (Text Input)",
                placeholder="Paste a loan application summary here...",
                lines=6
            )

            image_input = gr.Image(
                label="Loan Application (Image Input)",
                type="filepath"
            )

            question_input = gr.Textbox(
                label="Regulatory Question",
                placeholder="e.g., What supervisory concerns would FDIC examiners consider for this loan?",
                lines=3
            )

            submit_btn = gr.Button("Analyze Loan Application", variant="primary")

        with gr.Column():
            structured_output = gr.Textbox(
                label="Extracted Loan Information",
                lines=18,
                interactive=False
            )

            regulatory_output = gr.Textbox(
                label="FDIC Regulatory Analysis (Question-Based)",
                lines=20,
                interactive=False
            )

    submit_btn.click(
        fn=process_user_input,
        inputs=[text_input, image_input, question_input],
        outputs=[structured_output, regulatory_output]
    )

demo.launch()



/tmp/ipython-input-3605832963.py:2: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://598f7cf2fb403b7ed4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#Key Takeaways

* Prompt engineering alone can enforce regulatory alignment and safety

* Clear constraints prevent hallucination and decision-making

* Question-focused prompting improves faithfulness and accuracy

* Large context windows support audit-ready regulatory reasoning

* Proper refusal handling is critical in regulated domains